# Item Collaborative Filtering
* This notebook implements both item-based and user-based collaborative filtering
* Prediction is $\tilde r_{ij} = \dfrac{\sum_{k \in N(j)} w_{kj}^{\lambda_w}r_{ik}^{\lambda_r}}{\sum_{k \in N(j)} w_{kj}^{\lambda_w} + \lambda}$ for item-based collaborative filtering
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{kj}$ is the similarity between items $j$ and $k$
* $N(j)$ is the largest $K$ sorted by $w_{kj}$
* $\lambda_w, \lambda_r, \lambda$ are regularization parameters

In [1]:
source = "ItemCollaborativeFiltering";

In [2]:
using LinearAlgebra
using Memoize
using SparseArrays
# TODO upstream imports

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
#TODO in the original handle cases where an item is in validation but not in training

## Determine the neighborhoods for each user and item

In [5]:
function read_similarity_matrix(outdir)
    read_params(outdir)["S"]
end;

In [6]:
function get_abs_neighborhood(item, S, K)
    weights = S[:, item]
    K = Int(min(K, length(weights) - 1))
    # skip the most correlated item because it's always itself
    order = partialsortperm(abs.(weights), 2:K+1, rev = true)
    order, weights[order]
end;

In [7]:
# each prediction is just the weighted sum of all items in the neighborhood
# we apply regularization terms to decay the weights, ratings, and final prediction
function make_prediction(item, users, R, get_neighborhood, λ)
    items, weights = get_neighborhood(item)
    decay(x, a) = x != 0 ? sign(x) * abs(x)^a : 0
    weights = decay.(weights, λ[1])
    predictions = zeros(eltype(weights), length(users))
    weight_sum = zeros(eltype(weights), length(users))
    for u = 1:length(users)
        for (i, weight) in zip(items, weights)
            if R[users[u], i] != 0
                predictions[u] += weight * decay(R[users[u], i], λ[2])
                weight_sum[u] += abs(weight)
            end
        end
    end
    for u = 1:length(users)
        if weight_sum[u] + λ[3] != 0
            predictions[u] /= (weight_sum[u] + λ[3])
        end
    end
    predictions
end;

In [8]:
function collaborative_filtering(training, inference, get_neighborhood, λ)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        num_items(),
    )

    preds = zeros(eltype(λ), length(inference.rating), Threads.nthreads())
    @tprogress Threads.@threads for item in collect(Set(inference.item))
        mask = inference.item .== item
        preds[mask, Threads.threadid()] =
            make_prediction(item, inference.user[mask], R, get_neighborhood, λ)
    end

    vec(sum(preds, dims = 2))
end;

## Item based CF

In [9]:
cf_alphas = [x for x in read_params("CombineSignals")["alphas"] if occursin("ItemCF", x)]

4-element Vector{String}:
 "ItemCF.16"
 "ItemCF.64"
 "ItemCF.256"
 "ItemCF.1024"

In [10]:
for alpha in cf_alphas
    params = read_params(alpha)
    S = read_similarity_matrix(params["S"])
    K = params["K"]
    neighborhood_types = Dict("abs" => get_abs_neighborhood)
    neighborhoods = i -> neighborhood_types[params["neighborhood_type"]](i, S, K)
    λ = params["λ"]
    training = get_recommendee_residuals(params["training_residuals"])
    training.user .= 1 # relabel ids so that recommendee -> 1
    inference = RatingsDataset(fill(1, num_items()), 1:num_items(), zeros(num_items()))
    predictions = collaborative_filtering(training, inference, neighborhoods, λ)
    model(items) = predictions[items]
    write_recommendee_alpha(model, outdir = alpha)
end

Progress: 100%|███████████████████████████| Time: 0:00:13 ( 0.82 ms/it)
Progress: 100%|███████████████████████████| Time: 0:00:16 ( 0.97 ms/it)
Progress: 100%|███████████████████████████| Time: 0:00:13 ( 0.79 ms/it)
Progress: 100%|███████████████████████████| Time: 0:00:21 ( 1.24 ms/it)
